# Workbook Contents

# Alignment by Entropy

Method:

Step 1:
- Split sentences by word
- Step through words by column then row and for each word calculate entropy for column and dataframe before and after a given action taken. Actions:
    - Insert column and move word left
    - Move all words in row right
- Output: intermediate dataframe with sentences in correct order and unique words in columns* but with some columns containing 1 instance of a word that need to be consolidated.

Step 2:
- Sweep through dataframe to align intermediate columns with matching words
- Output: Aligned sentences with potential of column to have 2 different words*

Step 3:
- In cases of equal number of instances of word in the same column, entropy testing for adjustments does not pick up an adjustment as being better.
- Final step searches columns for more than 1 unique value and splits a column if this is found.
- Output: aligned sentences (not perfect there will still be some words that should appear in the same column)

# Optimal Text Selection

Step 4:
- Go through each column and check how frequent the word is (word count / no. rows)
Where frequency ratio is higher than given cutoff ratio select word

# Word Confidence for a given hypothesis

For a given hypothesis how confidence are we that the word is correct?
- Calculate percentage confidence of words for given position


## Imports

In [1]:
import os
import regex as re
import string
import itertools

import numpy as np
from numpy import mean
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

import pickle

import jiwer
from jiwer import wer

from tqdm import tqdm

In [2]:
print(os.getcwd())

/Users/traceyetheridge/Documents/Project/Git_Shared/20_alignment_methods


In [3]:
df_import = pd.read_pickle("/Users/traceyetheridge/Documents/Project/Git_Shared/data/df_clean_newwer_new.pkl")
df_import.shape

(333459, 10)

In [4]:
df_import.to_csv(r'/Users/traceyetheridge/Documents/Project/Git_Shared/data/df_clean_newwer_new.csv')

## Functions - Alignment

These are the functions that move words within the dataframe

In [5]:
def make_split_sentences_df(df):
    df = pd.DataFrame([sent.split() for sent in df])
    df = df.replace(np.nan, '', regex=True)
    return(df)

def move_word_left(df, col, row):
    df_temp = df.copy()
    cur_col_pos = df_temp.columns.get_loc(col)
    prior_col_pos = cur_col_pos - 1
    prior_col_name = df_temp.columns[prior_col_pos]
    
    if df_temp[prior_col_name][row] == "":
        col_words = df_temp[prior_col_name].values.tolist()
        col_words = list(filter(None, col_words))
        if df_temp[col][row] in col_words:
            df_temp[prior_col_name][row] = df_temp[col][row]
            df_temp[col][row] = ""
    return df_temp

def insert_col_left_move(df, col, row):
    df_temp = df.copy()
    cur_col_pos = df_temp.columns.get_loc(col)
    cur_col_name = df_temp.columns[cur_col_pos]
    prior_col_pos = cur_col_pos - 1
    prior_col_name = df_temp.columns[prior_col_pos]
    if prior_col_pos == -1:
        new_col_name = -1
    else:
        new_col_name = (cur_col_name - prior_col_name)/2 + prior_col_name
    
    df_temp[new_col_name] = ""
    df_temp = df_temp.reindex(sorted(df_temp.columns), axis=1)
    df_temp[new_col_name][row] = df_temp[col][row]
    df_temp[col][row] = ""
    return df_temp

def move_word_right_all(df,col,row):
    df_temp = df.copy()
    col_names = df_temp.columns.tolist()
    col_names_old = [i for i in col_names if i >= col]
    col_names_new = [i for i in col_names if i > col]
    col_names_new.append(df_temp.columns[-1] + 1)
    
    for col in reversed(col_names_new):
        if col not in df_temp.columns.tolist():
            df_temp[col] = ""
        df_temp[col][row] = df_temp[col-1][row]
        
    df_temp[col-1][row] = ""
    return df_temp

## Functions - Shannon Entropy Calculation

These calculate shannon entropy for columns or dataframe

In [6]:
from math import log2

def shannon(words):
    words = list(filter(None, words))
    counts = dict()
    for i in words:
        counts[i] = counts.get(i, 0) + 1
    
    total = sum(counts.values())
    entropy_calc = sum(freq / total * log2(total / freq) for freq in counts.values())
    return entropy_calc

# Entropy calc over column

def shannon_col(df, col):
    shannon_calc = shannon(df[col])
    return shannon_calc

# Entropy over data frame

def shannon_total(df):
    total_se = 0
    for col in range(len(df.columns)):
        total_se += shannon(df.iloc[:,col])
    return total_se

## Functions - Sweeping

After words have been moved based on entropy calculation these functions are used to move some words which were too far from a match to be aligned previously

In [7]:
def sweep_forward(df):
    df = df.copy()
    df.columns = range(df.shape[1])
    for col in range(len(df.columns)-2,-1, -1):

        for row in range(len(df)):

            if len(df[col][row]) > 0:
                if len(df[col+1][row]) == 0:

                    col_names = df.columns.tolist()
                    col_names_to_end = [i for i in col_names if i > col]

                    for col2 in col_names_to_end:
                        if df[col2][row] != "":
                            break
                        col_words = df[col2].values.tolist()

                        if df[col][row] in col_words:
                            df[col2][row] = df[col][row]
                            df[col][row] = ""
        
    # Delete empty columns                        
    df = df.replace('', np.nan)
    df = df.dropna(axis=1, how='all')
    df = df.replace(np.nan,'')
    df.columns = range(df.shape[1])
    
    return df

In [8]:
def sweep_back(df):
    df = df.copy()
    df.columns = range(df.shape[1])
    for col in range(1,len(df.columns)):

        for row in range(len(df)):

            if len(df[col][row]) > 0:
                if len(df[col-1][row]) == 0:

                    col_names = df.columns.tolist()
                    col_names_to_start = list(reversed([i for i in col_names if i < col]))

                    for col2 in col_names_to_start:
                        if df[col2][row] != "":
                            break
                        col_words = df[col2].values.tolist()

                        if df[col][row] in col_words:
                            df[col2][row] = df[col][row]
                            df[col][row] = ""
        
    # Delete empty columns                        
    df = df.replace('', np.nan)
    df = df.dropna(axis=1, how='all')
    df = df.replace(np.nan,'')
    df.columns = range(df.shape[1])
    
    return df

## Step 1 - Alignment with Shannon Entropy

In [9]:
def align_words(df):

    for col in range(len(df.columns)):

        for row in range(len(df)):

            if col != 0:
                df = move_word_left(df,col,row)

            # Calc entropy pre adjustment
            shannon_col_pre_adj = shannon_col(df, col)
            shanonn_df_pre_adj = shannon_total(df)

            # Calc entropy post adjustment
            shannon_df_insert_left = shannon_total(insert_col_left_move(df,col,row))
            shannon_df_all_move_right = shannon_total(move_word_right_all(df,col,row))
            shannon_col_all_move_right = shannon_col(move_word_right_all(df,col,row), col)

            if (shanonn_df_pre_adj > shannon_df_insert_left) or (shanonn_df_pre_adj > shannon_df_all_move_right):

                if shannon_df_insert_left <= shannon_df_all_move_right:
                    df = insert_col_left_move(df,col,row)
                else:
                    if shannon_col_all_move_right <= shannon_col_pre_adj:
                        df = move_word_right_all(df,col,row)

            # Calc entropy post adjustment
            shannon_col_post_adj = shannon_col(df, col)
            shannon_df_post_adj = shannon_total(df)

    # Delete empty columns 
    df = df.replace('', np.nan)
    df = df.dropna(axis=1, how='all')
    df = df.replace(np.nan,'')
    df.columns = range(df.shape[1])
    return df

## Step 2 - Alignment with sweeping

In [10]:
def sweep_combined(df):
    while not sweep_forward(df).equals(df):
        df = sweep_forward(df)

    while not sweep_back(df).equals(df):
        df = sweep_back(df)

    while not sweep_forward(df).equals(df):
        df = sweep_forward(df)
    
    return df

## Step 3 - Split any column with more than 1 word

Occurs when there is an equal number of instances of word

In [11]:
def split_column(df):

    for col in range(len(df.columns)):
        words_all = df[col]
        words_unique = words_all.unique()
        words_unique = list(filter(None, words_unique))
        if len(words_unique) > 1:
            new_col_name = col + 0.5
            df[new_col_name] = ""
            for row in range(len(words_all)):
                if df[col][row] == words_unique[1]:
                    df[new_col_name][row] = df[col][row]
                    df[col][row] = ""

    df = df.reindex(sorted(df.columns), axis=1)
    df.columns = range(df.shape[1])
    return df

## Step 4 -  Optimal Text Selection based on word Frequency

3/16 = 0.1875
4/16 = 0.2500
5/16 = 0.3125
6/16 = 0.3750

In [12]:
def hypothesis_merging(df):
    merged_hypothesis = []
    for col in range(df.shape[1]):
        words = df[col]
        words_not_empty = list(filter(None, words))
        word_count = len(words_not_empty)
        word_freq_ratio = word_count/len(df)
        if word_freq_ratio > 0.35:
            word_selected = words_not_empty[0]
            merged_hypothesis.append(word_selected)
    merged_hypothesis = ' '.join(word for word in merged_hypothesis)
    return merged_hypothesis

# Combined Function

In [13]:
def alignment_all(df):
    df = df.copy()
    
    df_align = pd.DataFrame(columns=["corpus", "reference.text", "alignment_text", "old_best_score", "new_wer_score", 
                                        "confidence",])

    identifiers = df['identifier'].unique()

    for identifier in tqdm(identifiers[:]):
        print('identifier: ',identifier)
        df_temp = df[df['identifier'] == identifier]
        df_sentences = df_temp['hypothesis.text']
        df_sentences = make_split_sentences_df(df_sentences)

        # Run one set of hypothesis texts through alignment functions
        df_sentences = align_words(df_sentences)
        df_sentences = sweep_combined(df_sentences)
        df_sentences = split_column(df_sentences)
        alignment_text = hypothesis_merging(df_sentences)

        ref_text = df_temp['reference.text'].iloc[0]
        best_score = df_temp["scoring.wer"].min()
        calculated_score = wer(ref_text, alignment_text)
        corpus = df_temp['corpus'].iloc[0]
        
        # Append results to dataframe
        df_align = df_align.append({
            'reference.text': ref_text,
            "alignment_text": alignment_text, 
            "old_best_score":best_score,
            "new_wer_score":calculated_score,
            'corpus': corpus
        },ignore_index=True
        )
    return df_sentences

In [14]:
#alignment_all(df_import)

# Example

Sentences:
- 2036: yeah ah which i think you know the the s the the service is is a big part of but but uh
- 2015: well when uh my my doggy uh that we have now uh she's she's mostly golden retriever
- 2013: i got a degree in in uh i got a business degree in college and that's the most useless degree in the world it's like why did they do this to people and um so you have to you know you have to"
- 953: and it for the one that's working with me it's working uh very well i don't have any problem with that i have one that's um helping me out at um for the litigation we're doing

In [15]:
df_test = df_import[df_import['identifier'] == df_import['identifier'].iloc[2013]]
df_test = df_test.drop(['file', "speaker_id"], axis=1)
print(len(df_test))
df_test.head(3)

13


,index,identifier,corpus,configuration,machine,reference.text,hypothesis.text,scoring.wer
2013,3235,sw4084A-ms98-a-0036,switchboard_segmented,amazon__8000_8,amazon,i got a degree in in uh i got a business degre...,i got a degree in in a i got a business degree...,0.046512
3100,3235,sw4084A-ms98-a-0036,switchboard_segmented,amazon_enus__8000_8,amazon,i got a degree in in uh i got a business degre...,i got a degree in in a i got a business degree...,0.046512
7121,3235,sw4084A-ms98-a-0036,switchboard_segmented,ms_azure_continuous__am_generic__lm_generic__e...,ms,i got a degree in in uh i got a business degre...,i got a degree in in i got a business degree i...,0.069767


In [16]:
sample = make_split_sentences_df(df_test['hypothesis.text'])
sample

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,i,got,a,degree,in,in,a,i,got,a,...,um,so,you,have,to,you,know,you,have,to
1,i,got,a,degree,in,in,a,i,got,a,...,um,so,you,have,to,you,know,you,have,to
2,i,got,a,degree,in,in,i,got,a,business,...,so,you,have,to,you,know,you,have,to,
3,i,got,a,degree,in,in,i,got,a,business,...,so,you,have,to,you,know,you,have,to,
4,i,got,a,degree,in,in,a,i,got,a,...,so,you,have,to,you,know,you,have,to,
5,i,got,a,degree,in,in,uh,i,got,a,...,um,so,you,have,to,you,know,you,have,to
6,i,got,a,degree,in,in-app,i,got,a,business,...,you,have,to,you,have,to,,,,
7,i,got,a,degree,in,in-app,i,got,a,business,...,you,have,to,you,have,to,,,,
8,i,got,a,degree,and,in,a,i,got,a,...,so,you,have,to,you,know,you,have,to,
9,i,got,a,degree,in,an,i,got,a,business,...,so,you,have,to,you,know,you,have,to,


In [17]:
sample = align_words(sample)
pd.set_option('display.max_columns', 500)
print(sample.shape)
sample

(13, 97)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
0,i,got,a,,degree,,,,in,,in,,,,,a,i,got,a,business,degree,,in,,college,,and,,,,that's,,,,the,,,,most,,,,useless,,,,,,,degree,,,,in,,,,the,,,world,,,,it's,,like,,,why,do,they,do,,,this,,to,people,and,um,,,,,,so,you,have,to,you,,know,,you,have,to
1,i,got,a,,degree,,,,in,,in,,,,,a,i,got,a,business,degree,,in,,college,,and,,,,that's,,,,the,,,,most,,,,useless,,,,,,,degree,,,,in,,,,the,,,world,,,,it's,,like,,,why,do,they,do,,,this,,to,people,and,um,,,,,,so,you,have,to,you,,know,,you,have,to
2,i,got,a,,degree,,,,in,,in,,,,,,i,got,a,business,degree,,in,,college,,and,,,,that's,,,,the,,,,most,,,,useless,,,,,,,degree,,,,in,,,,the,,,world,,,,it's,,like,,,why,do,they,do,,,this,,to,people,and,,uh,,,,,so,you,have,to,you,,know,,you,have,to
3,i,got,a,,degree,,,,in,,in,,,,,,i,got,a,business,degree,,in,,college,,and,,,,that's,,,,the,,,,most,,,,useless,,,,,,,degree,,,,in,,,,the,,,world,,,,it's,,like,,,why,do,they,do,,,this,,to,people,and,,uh,,,,,so,you,have,to,you,,know,,you,have,to
4,i,got,a,,degree,,,,in,,in,,,,,a,i,got,a,business,degree,,in,,college,,and,,,,that's,,,,the,,,,most,,,,useless,to,,,,,,,,,,in,,,,the,,,world,,,,it's,,like,,,why,do,they,do,,,this,,to,people,,,,,,and,,so,you,have,to,you,,know,,you,have,to
5,i,got,a,,degree,,,,in,,in,uh,,,,,i,got,a,business,degree,,in,,college,,and,,,,that's,,,,the,,,,most,,,,useless,,,,,,,degree,,,,in,,,,the,,,world,,,,it's,,like,,,why,do,they,do,,,this,,to,people,and,,,um,,,,so,you,have,to,you,,know,,you,have,to
6,i,got,a,,degree,,,,in,,,,in-app,,,,i,got,a,business,degree,,in,,college,,and,,,,that's,,,,the,,,,most,,,,useless,,degrees,,,,,,,,,in,,,,the,,,world,,,,it's,,like,,,why,do,they,do,,,this,,to,people,and,,,,,,say,,you,have,to,you,have,,to,,,
7,i,got,a,,degree,,,,in,,,,in-app,,,,i,got,a,business,degree,,in,,college,,and,,,,that's,,,,the,,,,most,,,,useless,,degrees,,,,,,,,,in,,,,the,,,world,,,,it's,,like,,,why,do,they,do,,,this,,to,people,and,,,,,,say,,you,have,to,you,have,,to,,,
8,i,got,a,,degree,and,,,in,,,,,,,a,i,got,a,business,degree,,,,college,,and,,,,that's,,,,the,,,,most,,,,useless,,,dean,,,,,the,,,,war,,,,all,,,that's,,,,,like,what,,,do,they,do,to,,,the,,people,and,,,,am,,,so,you,have,to,you,,know,,you,have,to
9,i,got,a,,,,degree,,in,,,,,an,,,i,got,a,business,degree,college,,and,,that's,,the,,,,most,,,,useless,,,,de,,,,,,,grand,,,,,e,,,,wor,,,all,,,,as,,,,like,,,why,do,they,do,,it,,,to,people,and,,,,am,,,so,you,have,to,you,,know,,you,have,to


In [18]:
sample = sweep_combined(sample)

pd.set_option('display.max_columns', 500)
print(sample.shape)
sample

(13, 80)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
0,i,got,a,degree,,in,in,,,,a,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,,,degree,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,,um,,,so,you,have,to,you,know,you,have,to
1,i,got,a,degree,,in,in,,,,a,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,,,degree,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,,um,,,so,you,have,to,you,know,you,have,to
2,i,got,a,degree,,in,in,,,,,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,,,degree,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,uh,,,,so,you,have,to,you,know,you,have,to
3,i,got,a,degree,,in,in,,,,,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,,,degree,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,uh,,,,so,you,have,to,you,know,you,have,to
4,i,got,a,degree,,in,in,,,,a,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,to,,,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,,,,,so,you,have,to,you,know,you,have,to
5,i,got,a,degree,,in,in,uh,,,,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,,,degree,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,,um,,,so,you,have,to,you,know,you,have,to
6,i,got,a,degree,,,in,,in-app,,,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,degrees,,,,,,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,,,,say,,you,have,to,,,you,have,to
7,i,got,a,degree,,,in,,in-app,,,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,degrees,,,,,,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,,,,say,,you,have,to,,,you,have,to
8,i,got,a,degree,and,,in,,,,a,i,got,a,business,degree,,,,college,,and,,,that's,,,the,,,most,,,useless,,dean,,,,,the,,,,war,,,all,,that's,,,,,like,what,,do,they,do,to,,,the,,people,and,,,am,,so,you,have,to,you,know,you,have,to
9,i,got,a,degree,,,in,,,an,,i,got,a,business,degree,college,,and,,that's,,the,,,most,,,useless,,,de,,,,,grand,,,,,e,,,,wor,,all,,,as,,,,like,,why,do,they,do,,it,,,to,people,and,,,am,,so,you,have,to,you,know,you,have,to


In [19]:
sample_alignment = hypothesis_merging(sample)
print('Selected text:  ',sample_alignment)
print('\nReference text: ',df_test['reference.text'].iloc[0])
print('\nWER: ',round(wer(df_test['reference.text'].iloc[0], sample_alignment),2))

Selected text:   i got a degree in in a i got a business degree in college and that's the most useless degree in the world it's like why do they do this to people and so you have to you know you have to

Reference text:  i got a degree in in uh i got a business degree in college and that's the most useless degree in the world it's like why did they do this to people and um so you have to you know you have to

WER:  0.07


# Word Confidence for a given hypothesis

In [20]:
sample.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
0,i,got,a,degree,,in,in,,,,a,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,,,degree,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,,um,,,so,you,have,to,you,know,you,have,to
1,i,got,a,degree,,in,in,,,,a,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,,,degree,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,,um,,,so,you,have,to,you,know,you,have,to
2,i,got,a,degree,,in,in,,,,,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,,,degree,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,uh,,,,so,you,have,to,you,know,you,have,to
3,i,got,a,degree,,in,in,,,,,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,,,degree,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,uh,,,,so,you,have,to,you,know,you,have,to
4,i,got,a,degree,,in,in,,,,a,i,got,a,business,degree,,in,,college,,and,,,that's,,,the,,,most,,,useless,,,,to,,,,,,in,,,,,the,,,world,it's,,like,,why,do,they,do,,,this,,to,people,and,,,,,so,you,have,to,you,know,you,have,to


### Calculate percentage confidence of words for given position

In [21]:
sample_sentence = sample.iloc[1]
sentence = []
confidence_pct = []
word_confidence = []
for index,word in enumerate(sample_sentence):
    if word != "":
        confidence_pct =  sample[index].value_counts()[word] / len(sample)
        word_confidence.append([word, round(confidence_pct,2)])
    
df_word_conf = pd.DataFrame(word_confidence)
df_word_conf.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
0,i,got,a,degree,in,in,a,i,got,a,business,degree,in,college,and,that's,the,most,useless,degree,in,the,world,it's,like,why,do,they,do,this,to,people,and,um,so,you,have,to,you,know,you,have,to
1,1,1,1,1,0.62,0.92,0.46,1,1,1,1,1,0.62,0.77,0.77,0.77,0.77,0.77,0.77,0.38,0.69,0.85,0.85,0.69,1,0.92,0.85,1,1,0.85,0.92,1,1,0.23,0.77,1,1,1,0.85,0.85,1,1,1
